In [1]:
import pandas as pd
import script
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Airtable

In [3]:
airtable = script.load_airtable(
    "./Transit Data Quality Issues-Open Issues by Distrcit.csv"
)

## Catastrophic

In [4]:
cat_df = script.load_catastrophic_errors(
    "./feed_info_expired_feeds_2023-10-30T15_12_27.01382Z.xlsx"
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
cat_df

,service_name,district,uri,date
4,Victor Valley Transit,8,https://ontime.vvta.org/gtfs,2023-10-29
3,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-29
8,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-28
12,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-27
16,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-26
20,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-25
24,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-24
28,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-23
32,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-22
36,TRACER,10,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,2023-10-21


In [6]:
script.summarize_cat(cat_df, airtable)

,service_name,uri,# of days with expired feed
0,TRACER,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,14
1,Victor Valley Transit,https://ontime.vvta.org/gtfs,1


,service_name,uri,# of days with expired feed,gtfs_datasets,services,issue_type,description,airtable_ticket
0,TRACER,https://data.trilliumtransit.com/gtfs/tracy-ca-us/tracy-ca-us.zip,14,Tracy Schedule,TRACER,Expired Schedule Feed,City of Tracy: GTFS Schedule Feed Expired,Yes
1,Victor Valley Transit,https://ontime.vvta.org/gtfs,1,NaN,NaN,NaN,NaN,NaN


## Incompleteness

In [7]:
script.load_api_511("API511.xlsx")

These are Bay Area feeds that keep track of RT


,new
13,AC TRANSIT
22,Bay Area Rapid Transit
25,Caltrain
37,County Connection
40,Dumbarton Express Consortium
43,Emery Go-Round
46,FAST
52,Golden Gate Transit
55,Livermore Amador Valley Transit Authority
58,Marin Transit


In [8]:
airtable.loc[airtable.issue_type.str.contains('Completeness')]

,gtfs_datasets,services,issue_type,description,airtable_ticket
9,Bay Area 511 Union City Transit VehiclePositions,Union City Transit,GTFS Realtime Completeness Problem,"In the past 14 days, GTFS-RT Vehicle Positions is 37.21%, which is a reduction from 46.15% VP recorded on August 25, 2023. Also, September GTFS-RT Vehicle Positions activity has fluctuated from 31.41% to 41.67%.",Yes
23,"Humboldt Schedule,Humboldt Trip Updates,Humboldt Vehicle Positions",Humboldt Transit Authority,GTFS Realtime Completeness Problem,"As of October 4, Humboldt Schedule GTFS-RT TripUpdates and VehiclePositions is 0%. 72.03% TU and 67.72% VP were recorded on October 3, 2023.",Yes
29,"MVGO VehiclePositions Historic,MVGO TripUpdates Historic",MVGO,GTFS Realtime Completeness Problem,Since July 3 there have been zero trips accounted for in both the Vehicle Positions and Trip Updates for MVGO,Yes
31,Merced VehiclePositions,Merced The Bus,GTFS Realtime Completeness Problem,Transit Joint Powers Authority for Merced County: GTFS-RT issue: complete outage of GTFS-RT Vehicle Positions Data,Yes
32,"Metrolink Schedule,Metrolink TripUpdates,Metrolink VehiclePositions",Metrolink,GTFS Realtime Completeness Problem,"From October 1-20, Metrolink Schedule GTFS-RT TripUpdates and VehiclePositions was 0%. For the past two weeks, TU and VP is reporting 12.7% of trip data. However, on October 19, TripUpdates and VehiclePositions both increased to 72.04%.",Yes
38,PresidiGo VehiclePositions,Presidio GO Shuttle,GTFS Realtime Completeness Problem,Has dipped below 40% threshold of accounting for scheduled trips in the last 14 days as of 8/22.,Yes
40,San Diego Vehicle Positions,San Diego Metropolitan Transit System,GTFS Realtime Completeness Problem,"The system appears incapable of handling enough requests. Every hour there are at a couple of requests that return HTTP 500 responses which indicate a failure on the San Diego servers. In the past we tried downloading trip updates, but experienced worsening quality on the vehicle positions. We do not currently download trip update data because of this.",Yes
42,"Santa Clarita Schedule,Santa Clarita TripUpdates,Santa Clarita VehiclePositions",Santa Clarita Transit,GTFS Realtime Completeness Problem,"As of October 9, Santa Clarita Schedule GTFS-RT TripUpdates and VehiclePositions recorded 0% trip data. In the last 14 days, 16.57% TU and 16.44% VP data was reported. On October 8, 92.11% TU and 91.76% VP were previously reported.",Yes
43,"Santa Rosa CityBus Schedule,Santa Rosa CityBus Vehicle Positions",Santa Rosa CityBus,GTFS Realtime Completeness Problem,As of 9/29: Only 4.9% of trips have trip update messages.,Yes
45,"StanRTA Schedule,StanRTA TripUpdates,StanRTA Vehicle Positions",Stanislaus Regional Transit Authority,GTFS Realtime Completeness Problem,"StanRTA Schedule GTFS-RT TripUpdates is 23.22%, which is a reduction from 67.95% TU recorded on October 2, 2023. GTFS-RT VehiclePositions is 22.94%, which is a reduction from 66.79% VP recorded on October 2, 2023.",Yes


In [9]:
script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2023-10-30T15_10_42.697685Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2023-10-30T15_10_45.214288Z.xlsx",
    airtable,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Anaheim Resort Schedule,13.78,11.42,Anaheim Resort Schedule,Anaheim Resort Transportation,Service Accuracy,Anaheim Transportation Network: GTFS-Schedule issue: thousands of duplicate trips on Toy Story Line,Yes
1,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA
2,Bay Area 511 BART Schedule,OK,0.00,NA,NA,NA,NA,NA
3,Bay Area 511 Capitol Corridor Schedule,37.93,37.93,NA,NA,NA,NA,NA
4,Bay Area 511 Commute.org Schedule,0.00,0.00,NA,NA,NA,NA,NA
5,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA
6,Bay Area 511 Mission Bay Schedule,0.00,0.00,NA,NA,NA,NA,NA
7,Bay Area 511 Rio Vista Delta Breeze Schedule,0.00,0.00,NA,NA,NA,NA,NA
8,Bay Area 511 SFO AirTrain Schedule,0.00,0.00,NA,NA,NA,NA,NA
9,Bay Area 511 South San Francisco Shuttle Schedule,0.00,0.00,NA,NA,NA,NA,NA
